# Time Synchronization Tests
This notebook tests the time synchronization for all the sensors in the array. The time synchronization procedure is as follows:
 - We insert a patter in the clock of the event camera, the IMU, and the FLIR camera
 - We look for this pattern by looking at the timestamps in local clock
 - We compute the offset between the beginning of the pattern and the local clock

In [ ]:
import sys
from mcap.reader import make_reader
from mcap_ros2.decoder import DecoderFactory, Decoder
from event_camera_py import Decoder as ECDecoder
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from pathlib import Path
from event_camera_py import Decoder
from bag_reader_ros2 import BagReader


In [ ]:
# Global variables used all over the file
#BAG=Path("/data/high_altitude_test/trigger_test/ha_ec_2024-08-09-11-38-38/ha_ec_2024-08-09-11-38-38_0.mcap")
BAG=Path("/data/high_altitude_test/calibration_record/ha_ec_2024-08-09-14-43-32/ha_ec_2024-08-09-14-43-32_0.mcap")
TRIGGER_FREQ = 50 # Hz

# FLIR-related constants
FLIR_TOPIC_RAW = "/cam_sync/cam0/image_raw"
FLIR_TOPIC_INFO =  "/cam_sync/cam0/camera_info"
FLIR_TOPIC_META = "/cam_sync/cam0/meta"

# VNAV-related constants
VNAV_TOPIC_COMMON = "/vectornav/raw/common"

# EC-related constants
EC_TOPIC = "/event_camera/events"

# Golden diffs are obtained from the logic analyzer
GOLDEN_DIFFS = np.array([724.68224, 1306.859319, 1006.997834])*1e-3 # in s

# Get statistics from the bag
with open(BAG, "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])

    # Get a dictionnary of the channels
    ch = reader.get_summary().channels
    topics = {ch[idx].topic:idx for idx in ch}
    # pprint(topics)
    
    # Get channel for FLIR and number of messages
    stats = reader.get_summary().statistics

print(f"Statistics for bag {BAG.name}:")
pprint(stats)
print(f"\nList of topics:")
pprint(topics)

## Computation of offset for FLIR camera

In [ ]:
# Get number of channels for the FLIR camera
flir_channel_raw = topics[FLIR_TOPIC_RAW]
flir_channel_info = topics[FLIR_TOPIC_INFO]
flir_channel_meta = topics[FLIR_TOPIC_META]
flir_number_raw = stats.channel_message_counts[flir_channel_raw]
flir_number_info = stats.channel_message_counts[flir_channel_info]
flir_number_meta = stats.channel_message_counts[flir_channel_meta]

# Number of messages for both channels should be the same
assert (flir_number_raw == flir_number_info == flir_number_meta)

# Get timing data from meta messages
num_msgs = flir_number_meta
arr_camtime = np.zeros(num_msgs)

# timesyncin_arr = np.zeros(
with open(BAG, "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])

    factory = DecoderFactory()
    decoders = {}
    
    i = 0
    for schema, channel, encoded_msg in reader.iter_messages():
        if channel.id != flir_channel_meta:
            continue
        # Create decoder for the channel id if it does not exist
        if channel.id not in decoders:
            decoders[channel.id] = factory.decoder_for(channel.message_encoding, schema)
        if i == 0:
            # print(f"{channel.topic} {schema.name}: {encoded_msg}")
            pass
        # decode message
        decoder = decoders[channel.id]
        msg = decoder(encoded_msg.data)

        arr_camtime[i] = msg.camera_time
        if (i % 500 == 0):
            print(f"{i} - {i/num_msgs}")
        i+=1
    arr_camtime/=1e9 # In s

In [ ]:
# Calculate the sequence times for the camera
diff_ts = np.diff(arr_camtime-arr_camtime[0])
p99 = np.percentile(diff_ts, 99)
assert np.isclose(1/TRIGGER_FREQ, p99, atol=1e-6, rtol=1e-4)

# Calculate the indices of the peaks
peak_idx = np.where(diff_ts > 2*1/TRIGGER_FREQ)[0]
assert len(peak_idx) == 4

# The peak corresponds to the last sample before and after the silence
time_diffs_flir = np.diff(arr_camtime[peak_idx])

diff_with_golden_flir = time_diffs_flir - GOLDEN_DIFFS
print(f"Golden diffs: {GOLDEN_DIFFS}")
print(f"This bag diffs: {time_diffs_flir}")
print(f"Diff with golden in us: {diff_with_golden_flir*1e6}")

# Timing is good when is less than half a ms
assert(np.all(diff_with_golden_flir*1e6<500))

# Time offset is the time of the first diff
time_offset = arr_camtime[peak_idx[0]]
print(f"Time offset: {time_offset}")

# Print timestamp of the first sample to double check
print(f"Timestamp of first sample: {arr_camtime[peak_idx[0]] - time_offset}") 

# Plot the time differencies
plt.figure()
plt.plot(np.diff(arr_camtime))
plt.show()

## Computation of offset for IMU

In [ ]:
vnav_channel_common = topics[VNAV_TOPIC_COMMON]
num_msgs = stats.channel_message_counts[vnav_channel_common]
arr_syncincnt = np.zeros(num_msgs)
arr_timesyncin = np.zeros(num_msgs) #ns
arr_timestartup = np.zeros(num_msgs)
arr_timegps = np.zeros(num_msgs)
arr_timegpspps = np.zeros(num_msgs)

with open(BAG, "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])

    factory = DecoderFactory()
    decoders = {}
    
    i = 0
    for schema, channel, encoded_msg in reader.iter_messages():
        if channel.id != vnav_channel_common:
            continue
        # Create decoder for the channel id if it does not exist
        if channel.id not in decoders:
            decoders[channel.id] = factory.decoder_for(channel.message_encoding, schema)
        if i == 0:
            # print(f"{channel.topic} {schema.name}: {encoded_msg}")
            pass
        # decode message
        decoder = decoders[channel.id]
        msg = decoder(encoded_msg.data)

        arr_syncincnt[i] = msg.syncincnt
        arr_timesyncin[i] = msg.timesyncin
        arr_timestartup[i] = msg.timestartup
        arr_timegps[i] = msg.timegps
        arr_timegpspps[i] = msg.timegpspps
        if (i % 2500 == 0):
            print(f"{i} - {i/num_msgs}")
        i+= 1
    arr_timesyncin /= 1e9 #us
# General plot of the syncincounter
#plt.figure()
#plt.plot(arr_timesyncin/5e3)
#plt.plot(arr_syncincnt)
#plt.show()

In [ ]:
# The VN is running at 400 Hz, and the sync pulse happens every 50 Hz. This means that there are 
# 8 samples in which syncincnt does not increase.
# It also means that the maximum timesyncin difference is 1/50 if we have a constant set of pulses

# Find the samples that have more than 1/50 for arr_timesyncin
greater_than_trigger_period = np.where(arr_timesyncin > 1/TRIGGER_FREQ)[0]

# Find when we have a discontinuity indicating gaps. The +1 is because the we are detecting the last stable sample of a group
gaps = np.where(np.diff(greater_than_trigger_period) > 1)[0] + 1

# Also add the first one 
gaps_idx = np.concatenate((np.array([greater_than_trigger_period[0]]), greater_than_trigger_period[gaps]))

# gaps_idx correspond to 8 samples after the trigger happened. Correct this offset
gaps_idx -= 8

# Verify that these indices have timesyncin that is less than 1/8 of the period of the trigger
# print(arr_timesyncin[gaps_idx])
assert np.all(arr_timesyncin[gaps_idx] < (1/8*1/TRIGGER_FREQ))

# Get all the sensor times for the gaps
time_diffs_imu = np.diff(arr_timestartup[gaps_idx]/1e9 - arr_timesyncin[gaps_idx])
diff_with_golden_imu = time_diffs_imu - GOLDEN_DIFFS
print(f"Golden diffs: {GOLDEN_DIFFS}")
print(f"This bag diffs: {time_diffs_imu}")
print(f"Diff with golden in us: {diff_with_golden_imu*1e6}")

# Get the first sample
first_sample_idx = gaps_idx[0]

# Get time offset
time_offset = arr_timestartup[first_sample_idx]/1e9 - arr_timesyncin[first_sample_idx]
print(f"Time offset: {time_offset}")

# Timestamp of the first sample
print(f"Timestamp of first sample: {arr_timestartup[first_sample_idx]/1e9 - time_offset}") 
print(f"timesyncin of first sample: {arr_timesyncin[first_sample_idx]}")

In [ ]:
plt.figure()
fig, ax = plt.subplots(nrows=2, ncols=2)
for i, idx in enumerate(gaps_idx):
    ax[i//2, i%2].plot(np.arange(idx-1, idx+10), arr_timesyncin[idx-1:idx+10])
    ax[i//2, i%2].plot(idx, arr_timesyncin[idx], 'or')
plt.show()


## Computation of offset Event Camera

In [ ]:
# Read events using Bernd's code
topic = EC_TOPIC
bag = BagReader(BAG, topic)
decoder = ECDecoder()
# Use the flir trigger to get the event numbers. These should be roughly the same * 2 due to polarity
triggers_ec = np.zeros(stats.channel_message_counts[flir_channel_meta])

i = 0
while bag.has_next():
    topic, msg, t_rec = bag.read_next()
    decoder.decode(msg)
    # cd_events = decoder.get_cd_events()
    # print(cd_events)
    trig_events = decoder.get_ext_trig_events()
    if len(trig_events) > 0 and trig_events[0][0] == 1:
        assert len(trig_events) == 1
        triggers_ec[i] = trig_events[0][1]
        i += 1
        # We hope the sequence is at the beginning
        if i >= stats.channel_message_counts[flir_channel_meta]:
            break
triggers_ec = triggers_ec[:i] / 1e6

In [ ]:
# Calculate the offsets for the event camera
diffs_ec = np.diff(triggers_ec)
p99 = np.percentile(diffs_ec, 99)
assert np.isclose(1/TRIGGER_FREQ, p99, atol=1e-6, rtol=1e-4)

# Calculate the indices of the peaks
peak_idx = np.where(diffs_ec > 2*1/TRIGGER_FREQ)[0]
assert len(peak_idx) == 4

# The peak corresponds to the last sample before and after the silence
time_diffs_ec = np.diff(triggers_ec[peak_idx])

diff_with_golden_ec = time_diffs_ec - GOLDEN_DIFFS
print(f"Golden diffs: {GOLDEN_DIFFS}")
print(f"This bag diffs: {time_diffs_ec}")
print(f"Diff with golden in us: {diff_with_golden_ec*1e6}")

# Timing is good when is less than half a ms
assert(np.all(diff_with_golden_ec*1e6<500))

# Time offset is the time of the first diff
time_offset = triggers_ec[peak_idx[0]]
print(f"Time offset: {time_offset}")

# Print timestamp of the first sample to double check
print(f"Timestamp of first sample: {triggers_ec[peak_idx[0]] - time_offset}") 

# Plot the time differencies
plt.figure()
plt.plot(np.diff(triggers_ec))
plt.show()

In [ ]:
# Compare the timings for all the sensors
print(f"EC: {diff_with_golden_ec*1e6}")
print(f"FLIR: {diff_with_golden_flir*1e6}")
print(f"IMU: {diff_with_golden_imu*1e6}")

In [ ]:
# Check the timestamps in the event camera
np.min(np.diff(triggers_ec[1000:]))